<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/transient_multiphase_flow_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Mathematical Foundation

### Drift-Flux Model

The core of the model is the Zuber-Findlay drift-flux relation:

$$v_G = C_0 \cdot v_m + v_d$$

Where:
- $v_G$ = gas velocity (m/s)
- $C_0$ = distribution coefficient (1.0-1.5, flow regime dependent)
- $v_m$ = mixture velocity (m/s)
- $v_d$ = drift velocity (m/s, depends on flow regime)

### Conservation Equations

The model solves the conservation equations for:

**Mass conservation (gas):**
$$\frac{\partial (\rho_G \alpha_G)}{\partial t} + \frac{\partial (\rho_G \alpha_G v_G)}{\partial x} = 0$$

**Mass conservation (liquid):**
$$\frac{\partial (\rho_L \alpha_L)}{\partial t} + \frac{\partial (\rho_L \alpha_L v_L)}{\partial x} = 0$$

**Momentum (mixture):**
$$\frac{\partial (\rho_m u)}{\partial t} + \frac{\partial (\rho_m u^2 + P)}{\partial x} = -\rho_m g \sin\theta - \frac{f \rho_m u^2}{2D}$$

Where:
- $\alpha_G, \alpha_L$ = gas and liquid volume fractions (holdup)
- $\rho_G, \rho_L, \rho_m$ = gas, liquid, and mixture densities
- $P$ = pressure
- $f$ = friction factor (Moody correlation)
- $\theta$ = pipe inclination angle
- $D$ = pipe diameter

### Flow Regime Detection

The model uses mechanistic criteria:

**Taitel-Dukler (1976)** for horizontal/near-horizontal:
- Stratified-Slug transition: Kelvin-Helmholtz stability
- Uses Lockhart-Martinelli parameter $X$

**Barnea (1987)** for inclined pipes:
- Unified model for all inclinations
- Bubble-slug transition at $\alpha_G \approx 0.25$
- Annular at high gas velocities

### Numerical Method

- **Spatial discretization:** Finite volume with AUSM+ flux scheme
- **Time integration:** Explicit Euler with adaptive time stepping
- **Stability:** CFL condition with $CFL = 0.5$ (default)

## 2. Setup and Basic Usage

First, let's import NeqSim and create a simple two-phase system.

In [ ]:
import neqsim
from neqsim.thermo import system, stream
from neqsim.process.equipment import pipeline

# Create a two-phase fluid system (methane + n-pentane)
fluid = system.SystemSrkEos(300.0, 50.0)  # 300 K, 50 bar
fluid.addComponent("methane", 0.8)  # 80 mol% methane
fluid.addComponent("n-pentane", 0.2)  # 20 mol% n-pentane
fluid.setMixingRule("classic")
fluid.setMultiPhaseCheck(True)
fluid.init(0)

print(f"System pressure: {fluid.getPressure():.2f} bar")
print(f"System temperature: {fluid.getTemperature():.2f} K")
print(f"Number of phases: {fluid.getNumberOfPhases()}")

## 3. Example 1: Horizontal Pipeline

Let's simulate a horizontal pipeline with constant flow rate.

In [ ]:
# Create inlet stream
inlet = stream.Stream("inlet", fluid.clone())
inlet.setFlowRate(5.0, "kg/sec")
inlet.setTemperature(300.0, "K")
inlet.setPressure(50.0, "bara")
inlet.run()

print(f"Inlet flow rate: {inlet.getFlowRate('kg/sec'):.2f} kg/s")
print(f"Inlet temperature: {inlet.getTemperature('K'):.2f} K")
print(f"Inlet pressure: {inlet.getPressure('bara'):.2f} bara")

In [ ]:
# Create transient pipe
pipe = pipeline.TransientPipe("HorizontalPipe", inlet)
pipe.setLength(1000.0)  # 1000 meters
pipe.setDiameter(0.2)  # 200 mm (0.2 m)
pipe.setRoughness(0.00005)  # 50 micrometers
pipe.setNumberOfSections(50)  # Discretize into 50 cells
pipe.setMaxSimulationTime(60.0)  # Simulate 60 seconds
pipe.setCflNumber(0.5)  # CFL number for stability

# Run simulation
print("Running transient pipe simulation...")
pipe.run()
print("Simulation complete!")

### Extract and Visualize Results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get spatial profiles
pressures = pipe.getPressureProfile()  # Pa
temperatures = pipe.getTemperatureProfile()  # K
holdups = pipe.getLiquidHoldupProfile()  # fraction
gas_velocities = pipe.getGasVelocityProfile()  # m/s
liquid_velocities = pipe.getLiquidVelocityProfile()  # m/s

# Create position array (m)
positions = np.linspace(0, pipe.getLength(), len(pressures))

# Convert pressure to bar
pressures_bar = [p / 1e5 for p in pressures]

print(f"Inlet pressure: {pressures_bar[0]:.2f} bar")
print(f"Outlet pressure: {pressures_bar[-1]:.2f} bar")
print(f"Pressure drop: {pressures_bar[0] - pressures_bar[-1]:.2f} bar")
print(f"Average liquid holdup: {np.mean(holdups):.3f}")

In [ ]:
# Plot results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Pressure profile
axes[0, 0].plot(positions, pressures_bar, 'b-', linewidth=2)
axes[0, 0].set_xlabel('Position (m)')
axes[0, 0].set_ylabel('Pressure (bar)')
axes[0, 0].set_title('Pressure Profile')
axes[0, 0].grid(True, alpha=0.3)

# Temperature profile
axes[0, 1].plot(positions, temperatures, 'r-', linewidth=2)
axes[0, 1].set_xlabel('Position (m)')
axes[0, 1].set_ylabel('Temperature (K)')
axes[0, 1].set_title('Temperature Profile')
axes[0, 1].grid(True, alpha=0.3)

# Liquid holdup
axes[1, 0].plot(positions, holdups, 'g-', linewidth=2)
axes[1, 0].set_xlabel('Position (m)')
axes[1, 0].set_ylabel('Liquid Holdup (-)')
axes[1, 0].set_title('Liquid Holdup Profile')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_ylim([0, 1])

# Velocities
axes[1, 1].plot(positions, gas_velocities, 'b-', linewidth=2, label='Gas')
axes[1, 1].plot(positions, liquid_velocities, 'r-', linewidth=2, label='Liquid')
axes[1, 1].set_xlabel('Position (m)')
axes[1, 1].set_ylabel('Velocity (m/s)')
axes[1, 1].set_title('Phase Velocities')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Example 2: Terrain Pipeline with Low Point

Now let's simulate a pipeline with terrain variation, which can lead to liquid accumulation and slugging.

In [ ]:
# Create new inlet stream
inlet2 = stream.Stream("inlet2", fluid.clone())
inlet2.setFlowRate(3.0, "kg/sec")
inlet2.setTemperature(300.0, "K")
inlet2.setPressure(60.0, "bara")
inlet2.run()

# Create terrain pipe
terrain_pipe = pipeline.TransientPipe("TerrainPipe", inlet2)
terrain_pipe.setLength(2000.0)  # 2000 meters
terrain_pipe.setDiameter(0.25)  # 250 mm
terrain_pipe.setNumberOfSections(40)
terrain_pipe.setMaxSimulationTime(300.0)  # 300 seconds

# Define elevation profile with a low point
n_sections = 40
elevations = []
for i in range(n_sections + 1):
    x = i * 2000.0 / n_sections
    if x < 500:
        # Horizontal
        elevations.append(0.0)
    elif x < 1000:
        # Downhill to -20m
        elevations.append(-20.0 * (x - 500) / 500)
    elif x < 1500:
        # Uphill back to 0
        elevations.append(-20.0 + 20.0 * (x - 1000) / 500)
    else:
        # Horizontal
        elevations.append(0.0)

terrain_pipe.setElevationProfile(elevations)

print("Running terrain pipe simulation...")
terrain_pipe.run()
print("Simulation complete!")

In [ ]:
# Extract results
positions2 = np.linspace(0, terrain_pipe.getLength(), len(terrain_pipe.getPressureProfile()))
pressures2 = [p / 1e5 for p in terrain_pipe.getPressureProfile()]
holdups2 = terrain_pipe.getLiquidHoldupProfile()
elevations_plot = np.array(elevations[:-1])  # Remove last node for cell-centered values

# Check accumulation zones
accum_tracker = terrain_pipe.getAccumulationTracker()
accum_zones = accum_tracker.getAccumulationZones()

print(f"Number of accumulation zones identified: {len(accum_zones)}")
for i, zone in enumerate(accum_zones):
    print(f"  Zone {i+1}: Position = {zone.getPosition():.1f} m, "
          f"Accumulated volume = {zone.getAccumulatedVolume():.2f} m³")

In [ ]:
# Plot terrain pipeline results
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Elevation profile
axes[0].fill_between(positions2, elevations_plot, -30, alpha=0.3, color='brown')
axes[0].plot(positions2, elevations_plot, 'k-', linewidth=2)
axes[0].set_ylabel('Elevation (m)')
axes[0].set_title('Pipeline Elevation Profile')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Mark accumulation zones
for zone in accum_zones:
    axes[0].axvline(x=zone.getPosition(), color='red', linestyle='--', alpha=0.7, linewidth=1.5)
    axes[0].text(zone.getPosition(), -25, 'Low Point', rotation=90,
                verticalalignment='bottom', color='red')

# Pressure profile
axes[1].plot(positions2, pressures2, 'b-', linewidth=2)
axes[1].set_ylabel('Pressure (bar)')
axes[1].set_title('Pressure Profile')
axes[1].grid(True, alpha=0.3)

# Liquid holdup
axes[2].plot(positions2, holdups2, 'g-', linewidth=2)
axes[2].set_xlabel('Position (m)')
axes[2].set_ylabel('Liquid Holdup (-)')
axes[2].set_title('Liquid Holdup Profile - Note Accumulation at Low Point')
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim([0, 1])

# Mark accumulation zones
for zone in accum_zones:
    axes[2].axvline(x=zone.getPosition(), color='red', linestyle='--', alpha=0.7, linewidth=1.5)

plt.tight_layout()
plt.show()

## 5. Example 3: Slug Tracking

The model includes Lagrangian slug tracking to capture individual slug units.

In [ ]:
# Access slug tracker
slug_tracker = terrain_pipe.getSlugTracker()

# Get slug statistics
n_slugs = slug_tracker.getSlugCount()
total_generated = slug_tracker.getTotalSlugsGenerated()
avg_length = slug_tracker.getAverageSlugLength()
slug_frequency = slug_tracker.getSlugFrequency()

print(f"Active slugs: {n_slugs}")
print(f"Total slugs generated: {total_generated}")
print(f"Average slug length: {avg_length:.2f} m")
print(f"Slug frequency: {slug_frequency:.4f} slugs/s")
print("\nDetailed statistics:")
print(slug_tracker.getStatisticsString())

## 6. Example 4: Vertical Riser

Vertical risers are important in offshore production. Let's simulate one.

In [ ]:
# Create riser inlet
riser_inlet = stream.Stream("riser_inlet", fluid.clone())
riser_inlet.setFlowRate(8.0, "kg/sec")
riser_inlet.setTemperature(300.0, "K")
riser_inlet.setPressure(80.0, "bara")
riser_inlet.run()

# Create vertical riser
riser = pipeline.TransientPipe("Riser", riser_inlet)
riser.setLength(200.0)  # 200 meters tall
riser.setDiameter(0.15)  # 150 mm
riser.setNumberOfSections(40)
riser.setMaxSimulationTime(120.0)

# Create vertical elevation profile
vertical_elevations = [i * 200.0 / 40 for i in range(41)]
riser.setElevationProfile(vertical_elevations)

print("Running riser simulation...")
riser.run()
print("Simulation complete!")

# Calculate results
riser_pressures = [p / 1e5 for p in riser.getPressureProfile()]
riser_holdups = riser.getLiquidHoldupProfile()
riser_positions = np.linspace(0, riser.getLength(), len(riser_pressures))

pressure_drop = riser_pressures[0] - riser_pressures[-1]
print(f"\nRiser pressure drop: {pressure_drop:.2f} bar")
print(f"Average liquid holdup: {np.mean(riser_holdups):.3f}")

In [ ]:
# Plot riser results
fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# Pressure vs height
axes[0].plot(riser_pressures, riser_positions, 'b-', linewidth=2)
axes[0].set_xlabel('Pressure (bar)')
axes[0].set_ylabel('Height (m)')
axes[0].set_title('Vertical Riser - Pressure Profile')
axes[0].grid(True, alpha=0.3)
axes[0].invert_xaxis()  # Higher pressure at bottom

# Holdup vs height
axes[1].plot(riser_holdups, riser_positions, 'g-', linewidth=2)
axes[1].set_xlabel('Liquid Holdup (-)')
axes[1].set_ylabel('Height (m)')
axes[1].set_title('Vertical Riser - Liquid Holdup')
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim([0, 1])

plt.tight_layout()
plt.show()

## 7. Flow Regime Analysis

The model automatically detects flow regimes. Let's examine them.

In [ ]:
# Get sections from horizontal pipe
sections = pipe.getSections()

# Extract flow regime information
regimes = []
regime_positions = []

for section in sections:
    regimes.append(section.getFlowRegime().toString())
    regime_positions.append(section.getPosition())

# Count regime occurrences
from collections import Counter
regime_counts = Counter(regimes)

print("Flow regime distribution:")
for regime, count in regime_counts.items():
    percentage = 100.0 * count / len(regimes)
    print(f"  {regime}: {count} sections ({percentage:.1f}%)")

## 8. Drift-Flux Parameters

Let's examine the drift-flux parameters calculated by the model.

In [ ]:
# Access drift-flux model
drift_flux = pipeline.DriftFluxModel()

# Calculate parameters for a representative section
mid_section = sections[len(sections) // 2]
params = drift_flux.calculateDriftFlux(mid_section)

print(f"Drift-flux parameters at position {mid_section.getPosition():.1f} m:")
print(f"  Distribution coefficient (C0): {params.C0:.3f}")
print(f"  Drift velocity (v_d): {params.driftVelocity:.3f} m/s")
print(f"  Void fraction (alpha_G): {params.voidFraction:.3f}")
print(f"  Slip ratio (S): {params.slipRatio:.3f}")
print(f"  Flow regime: {mid_section.getFlowRegime().toString()}")

## 9. Performance Tuning

### Key Parameters

1. **Number of sections**: More sections = better accuracy but slower
   - Recommended: 20-100 sections
   - Guideline: dx ≈ 10-50 pipe diameters

2. **CFL number**: Controls time step stability
   - Default: 0.5
   - Lower (0.3) = more stable, slower
   - Higher (0.8) = faster, less stable

3. **Thermodynamic update interval**: Flash calculation frequency
   - Default: every 10 time steps
   - Increase to speed up (e.g., 20)
   - Disable for isothermal: `setUpdateThermodynamics(false)`

In [ ]:
# Example: Fast simulation with reduced accuracy
fast_pipe = pipeline.TransientPipe("FastPipe", inlet.clone())
fast_pipe.setLength(1000.0)
fast_pipe.setDiameter(0.2)
fast_pipe.setNumberOfSections(20)  # Fewer sections
fast_pipe.setMaxSimulationTime(60.0)
fast_pipe.setCflNumber(0.7)  # Higher CFL
fast_pipe.setThermodynamicUpdateInterval(20)  # Less frequent flashes

import time
start_time = time.time()
fast_pipe.run()
elapsed_time = time.time() - start_time

print(f"Fast simulation completed in {elapsed_time:.2f} seconds")

## 10. Summary of Capabilities

### What Can Be Calculated:

1. **Spatial Profiles:**
   - Pressure, temperature, density
   - Liquid holdup (volume fraction)
   - Gas and liquid velocities
   - Flow regimes

2. **Transient Behavior:**
   - Time history of all variables
   - Startup/shutdown transients
   - Pressure wave propagation

3. **Slug Dynamics:**
   - Slug front and tail positions
   - Slug lengths and velocities
   - Slug frequency and statistics
   - Terrain-induced slugging

4. **Accumulation Zones:**
   - Location of liquid accumulation
   - Accumulated volumes
   - Overflow conditions

5. **Integration:**
   - Works with NeqSim ProcessSystem
   - Compatible with all thermodynamic models (SRK, PR, CPA)
   - Can be connected to separators, compressors, etc.

### Recommended Use Cases:

- **Terrain-induced slugging** prediction and mitigation
- **Riser design** and optimization
- **Startup/shutdown** procedure development
- **Liquid accumulation** analysis
- **Pressure drop** calculation with flow regime effects
- **Transient operational** scenarios

### References:

1. Zuber, N. and Findlay, J.A. (1965) - Drift-flux formulation
2. Taitel, Y. and Dukler, A.E. (1976) - Flow regime transitions
3. Barnea, D. (1987) - Unified inclination model
4. Bendiksen, K.H. (1984) - Slug dynamics